<a href="https://colab.research.google.com/github/shravanisci/ABC-Immune-Circuit/blob/main/Melanoma_ImTher_ABC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install GEOparse

In [ ]:
import os

os.makedirs("results", exist_ok=True)
print("Results directory ready.")


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request

plt.rcParams["figure.dpi"] = 120
os.makedirs("results_melanoma", exist_ok=True)

GSE_ID = "GSE165278"
print("Setup complete")


In [ ]:
import GEOparse

gse = GEOparse.get_GEO("GSE165278", destdir=".", silent=True)

print("Supplementary files:")
for f in gse.metadata.get("supplementary_file", []):
    print(f)


In [ ]:
import os
import urllib.request

url = "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE165nnn/GSE165278/suppl/GSE165278_TPM_original_23k_genes.xlsx"
os.makedirs("data", exist_ok=True)

out_path = "data/GSE165278_TPM.xlsx"
urllib.request.urlretrieve(url, out_path)

print("Downloaded:", out_path)


In [ ]:
import pandas as pd

expr_raw = pd.read_excel(out_path)
print("Raw shape:", expr_raw.shape)
expr_raw.head()


In [ ]:
# Identify gene column automatically
gene_col = expr_raw.columns[0]
print("Using gene column:", gene_col)

# Set gene names as index
expr = expr_raw.set_index(gene_col)

# Drop rows without gene names
expr = expr[expr.index.notna()]

# Standardize gene symbols
expr.index = expr.index.astype(str).str.upper()

# Transpose → rows = samples, columns = genes
expr = expr.T

print("Final expression matrix:", expr.shape)
expr.iloc[:5, :5]


In [ ]:
import numpy as np

def resolve_signal(df, candidates):
    found = [g for g in candidates if g in df.columns]
    if not found:
        return None
    return df[found].mean(axis=1)

signals = {
    "TNF": resolve_signal(expr, ["TNF"]),
    "IL6": resolve_signal(expr, ["IL6"]),
    "IFNG": resolve_signal(expr, ["IFNG"]),
    "IL10": resolve_signal(expr, ["IL10"])
}

signals = {k:v for k,v in signals.items() if v is not None}
signals_df = pd.DataFrame(signals)

print("Resolved cytokine matrix:", signals_df.shape)
signals_df.head()


In [ ]:
# Percentile thresholds regulatory circuit
thr = {
    "TNF": signals_df["TNF"].quantile(0.75),
    "IL6": signals_df["IL6"].quantile(0.75),
    "IFNG": signals_df["IFNG"].quantile(0.25),
}

abc_on_reg = (
    (signals_df["TNF"] > thr["TNF"]) &
    (signals_df["IL6"] > thr["IL6"]) &
    (signals_df["IFNG"] < thr["IFNG"])
).astype(int)

print("Regulatory ABC ON:", abc_on_reg.sum(), "/", len(abc_on_reg))

In [ ]:
# Percentile thresholds effector circuit (melanoma immunotherapy has ifng^)
thr = {
    "TNF": signals_df["TNF"].quantile(0.25),
    "IL6": signals_df["IL6"].quantile(0.25),
    "IFNG": signals_df["IFNG"].quantile(0.75),
}

abc_on_eff = (
    (signals_df["TNF"] > thr["TNF"]) &
    (signals_df["IL6"] > thr["IL6"]) &
    (signals_df["IFNG"] < thr["IFNG"])
).astype(int)

print("Effector ABC ON:", abc_on_eff.sum(), "/", len(abc_on_eff))


In [ ]:
import os
os.makedirs("results", exist_ok=True)

abc_on_eff.to_csv("results/GSE165278_ABC_effector.csv")
abc_on_reg.to_csv("results/GSE165278_ABC_regulatory.csv")


In [ ]:
import json

thresholds = {
    "TNF": "25th percentile",
    "IL6": "25th percentile",
    "IFNG": "75th percentile",
    "circuit": "ABC-Effector"
}

with open("results/GSE165278_thresholds_effector.json", "w") as f:
    json.dump(thresholds, f, indent=2)


In [ ]:

import os
os.listdir("results")

In [ ]:
import os
os.makedirs("results", exist_ok=True)

abc_on_reg.to_csv("results/MELANOMA_regulatory.csv")
abc_on_eff.to_csv("results/MELANOMA_effector.csv")

print(os.listdir("results"))
